# “网络数据采集” 课程

# 第4章 Web内容解析



## 案例名称

爬取新浪财经网http://finance.sina.com.cn/stock/，各股票公司每日公告（爬取股票分析所需语料）

## 案例内容

本案例用于爬取雪球网站股票实时信息。

基本流程如下：

1. 使用chrome开发者工具查看“沪深股市” -“沪深一栏”页面的url，大致如下：

![雪球股票实时案例1](images/04/雪球股票实时案例1.png)

```
https://xueqiu.com/service/v5/stock/screener/quote/list?page=1&size=30&order=desc&orderby=percent&order_by=percent&market=CN&type=sh_sz&_=1564104444269
```

分析查询参数为：

```
page: 1
size: 30
order: desc
orderby: percent
order_by: percent
market: CN
type: sh_sz
_: 1564104444269
```
参数 type指市场；最后一个参数"_"是按秒计的当前系统时间*1000，可用time.time()*1000获得，web系统不验证

2. 使用浏览器打开上述url，有以下内容：

![雪球股票实时案例2](images/04/雪球股票实时案例2.png)

显然是 json 格式数据。

3. 设计程序如下，用于访问url获取json数据。

默认爬取了5页股价信息，目前雪球上a股大致有126页信息。

结果保存在程序当前目录下的xueqiu.csv文件下。

In [6]:
import requests
import time

_type_ = "sh_sz"  #香港就填Hk # 沪深就填sh_sz，美股就选"US"

headers={
         "user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",
         "Cookie": "",   #上雪球把自己的cookie复制下来
         }

f = open("xueqiu.csv", "w")
f.write("symbol, name, current, percent, market_capital, pe_ttm\n")


def get_json(page):
    #带上cookie get请求
    return requests.get("https://xueqiu.com/service/v5/stock/screener/quote/list?page={}"
                        "&size=30&order=desc&orderby=percent&order_by=percent&market={}"
                        "&type={}&_={}".format(page, _type_, _type_, int(time.time()*1000)),
                        headers=headers).json()
    


def parse_json(data, _all):                          #解析数据
    _list = data["data"]["list"]
    for _each in _list:
        symbol = _each.get("symbol")                 #股票代码
        name = _each.get("name")                     #股票名称
        current = _each.get("current")               #当前价格
        percent = _each.get("percent")               #涨跌幅
        market_capital = _each.get("market_capital") #市值
        pe_ttm = _each.get("pe_ttm")                 #市盈率
        f.write(','.join(map(str, [symbol,name,current,percent,market_capital,pe_ttm])))
        f.write("\n")
        _all += 1
        if _all == 100:
            break
    return _all


def main():
    _all = 0
    for i in range(1,5):
        data = get_json(i)
        _all = parse_json(data, _all)

        time.sleep(5)                                #sleep 5s，


def test():
    print(requests.get("https://xueqiu.com/service/v5/stock/screener/quote/list?page=3&size=30&order=desc&orderby=percent&order_by=percent&market=US&type=us&_=1555485032232",
                       headers=headers).json())

if __name__ == "__main__":

    main()
    # test()

## 案例结论

这个案例较为综合的演示了：

- 使用requests库爬取雪球股票网站的实时数据

- 使用json解析文档内容，提取股价文本



